# Proyecto: Impacto del Riego en la Productividad Agrícola en México
## ETL reproducible
Este notebook contiene el flujo ETL base para preparar los datos.

In [5]:
import pandas as pd
import numpy as np
import os

## Funciones ETL

In [37]:
links_cap = ["https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_12.xlsx",
             "https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_agr00.xlsx",
             "https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_str_agr01.xlsx",
             "https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_str_agr04.xlsx",
             "https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_str_agr03.xlsx",
             "https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_agr06.xlsx",
             "https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_02.xlsx",
             "https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_02.xlsx",
             "https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_09.xlsx",
             "https://www.inegi.org.mx/contenidos/programas/ca/2022/tabulados/ca2022_agr11.xlsx"]

temp_dir = 'temp_csv'
os.makedirs(temp_dir, exist_ok=True)

# Crear archivos CSV para cada uno
paths = []
for link in links_cap:
    # Extraer el nombre del archivo del link y cambiar la extensión a .csv
    file_name = link.split("/")[-1].replace(".xlsx", ".csv")
    file_path = os.path.join(temp_dir, file_name)

    # Leer el archivo Excel desde el link y guardarlo como CSV
    try:
        df = pd.read_excel(link)
        headers = df[df.isnull().any(axis=1)].head(10)
        for col in headers.columns:
          print(headers[headers[col].notnull()][col].to_list())
        df = df.dropna()
        df.to_csv(file_path, index=False)
        paths.append(file_path)
        print(f"✓ Archivo creado: {file_name}")
    except Exception as e:
        print(f"✗ No se pudo procesar el link {link}: {e}")

print(f"\nTotal de archivos procesados: {len(paths)}")

0    Número de unidades de producción agrícola y po...
1        Datos de octubre de 2021 a septiembre de 2022
3                                   Entidad federativa
9                                               00 NAL
Name: INEGI. Censo Agropecuario 2022, dtype: object
3    Municipio
Name: Unnamed: 1, dtype: object
3    Entidad federativa y municipio
9          Estados Unidos Mexicanos
Name: Unnamed: 2, dtype: object
3    Unidades de producción agropecuaria activas
4                                          Total
8                                              A
9                                        4629134
Name: Unnamed: 3, dtype: object
4    Con actividad agrícola
5                     Total
8                      B<=A
9                   4422243
Name: Unnamed: 4, dtype: object
5    Con superficie de riego
6                      Total
8                       C<=B
9                     870853
Name: Unnamed: 5, dtype: object
6    A las que se les preguntó el tema de sistemas ...
8 

In [22]:
data_path = './temp_csv'
files = os.listdir(data_path)
files

['ca2022_agr06.csv',
 'ca2022_str_agr01.csv',
 'ca2022_12.csv',
 'ca2022_str_agr03.csv',
 'ca2022_str_agr04.csv',
 'ca2022_agr11.csv',
 'ca2022_09.csv',
 'ca2022_agr00.csv',
 'ca2022_02.csv']

In [23]:
def load_dataset(path):
    return pd.read_csv(path, encoding='utf-8', low_memory=False)

def clean_columns(df):
    df.columns = (
        df.columns
        .str.lower()
        .str.replace(' ', '_')
        .str.replace('[^a-z0-9_]', '', regex=True)
    )
    return df

def add_variables(df, surface_col='superficie', production_col='produccion'):
    if surface_col in df.columns and production_col in df.columns:
        df['rendimiento'] = df[production_col] / df[surface_col].replace(0, np.nan)
    return df

## Pipeline ejemplo (placeholders)

In [24]:
example_file = 0
for f in files:
    if f.endswith('.csv'):
        example_file = os.path.join(data_path, f)
        if example_file:
          print(example_file)
          df = load_dataset(example_file)
          df = clean_columns(df)
          df = add_variables(df)
          print(df.head())
        else:
            'No CSV files found to demonstrate ETL.'


./temp_csv/ca2022_agr06.csv
            unnamed_0        unnamed_1                             unnamed_2  \
0  Entidad federativa  Cultivo perenne  Entidad federativa y cultivo perenne   
1                 NaN              NaN                                   NaN   
2                 NaN              NaN                                   NaN   
3                 NaN              NaN                                   NaN   
4                 NaN              NaN                                   NaN   

                                     unnamed_3  \
0  Unidades de producción agropecuaria activas   
1                                          NaN   
2                                        Total   
3                                          NaN   
4                                          NaN   

                               unnamed_4  \
0                                    NaN   
1                                    NaN   
2  Con cultivos perennes a cielo abierto   
3             

In [20]:
df.describe()

,inegi_censo_agropecuario_2022,unnamed_1,unnamed_2,unnamed_3,unnamed_4,unnamed_5,unnamed_6,unnamed_7,unnamed_8,unnamed_9,cuadro_ca2022_02
count,38,36,36,35,36,36,35,36,35,35,35
unique,38,36,36,35,36,36,35,36,35,35,35
top,Superficie total y de uso agropecuario y fores...,Superficie del país,Superficie con uso o vocación agropecuaria y a...,Cubierta por el censo,Superficie declarada de las unidades de produc...,Uso agrícola,Sembrada u ocupada con cultivos,No sembrada,En descanso,"Agostadero, pastos naturales, enmontada, bosqu...",Con otros usos
freq,1,1,1,1,1,1,1,1,1,1,1
